# Show reference atlas slices

## Description

The mouse brain reference volumes are a pair of volumetric files from the Allen Institute for Brain Science (AIBS): the Nissl and the annotations volumes.

The Nissl volume shows every cell in the brain. It is used to obtain the cell distribution in the mouse brain. The annotations volume provides a brain region id to each voxel of the Nissl volume and any volumetric dataset of the AIBS. The mouse brain regions are organized in a hierarchical tree, with leaves representing the finest subdivisions captured by the AIBS. This hierarchy is stored in a json file and is common to each reference atlas version. It is also available on the AIBS website.

These two whole brain volumes exist in multiple versions and need to be aligned to each other to produce the best results in terms of cell density estimation (see https://doi.org/10.1101/2021.11.20.469384 for more details).

To test the alignment between the different pairs of annotations volume and Nissl volume, we are plotting the annotations volumes in color on top of the Nissl volumes.

In [ ]:
import numpy as np
import nrrd
import json
from voxcell import RegionMap, VoxelData
from os.path import join, isfile, realpath
from pylab import plt
import matplotlib.colors as colors

Change below the path to the folder containing the data, including the annotations volumes, the Nissl volumes and the json hierarchy file.

Change also the folder in which the figures produced will be stored.

There are different versions of the reference atlas pairs that we store in subfolders.

In [ ]:
DATA_FOLDER = 'path/to/data/'
OUTPUT_FOLDER = "/path/to/output/"
CCF_VERSION = "ccfv2"

In [ ]:
region_map = RegionMap.load_json(join(DATA_FOLDER, "brain_regions.json"))

In [ ]:
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, CCF_VERSION, "annotations.nrrd")).raw
nissl = VoxelData.load_nrrd(join(DATA_FOLDER, CCF_VERSION, "ara_nissl_25.nrrd")).raw

In [ ]:
sagittal_slice = int(ann.shape[2] // 2.5)

In [ ]:
cmap = plt.cm.gray
norm = plt.Normalize(nissl.min(), nissl.max())
rgba = cmap(1-norm(nissl[:, :, sagittal_slice].T))

rgba-=0.1
rgba = np.maximum(rgba/np.max(rgba), 0)

fig = plt.figure(figsize=(15, 9.1))
plt.imshow(rgba, interpolation='nearest')
plt.axis('off')
plt.xlim([0, annotation.shape[0]])
plt.ylim([annotation.shape[1], 0])
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "nissl_exp.png"), dpi=200, facecolor="white")

In [ ]:
np.random.seed(182) 
uniq = np.unique(annotation)
cmap = plt.cm.viridis 
cmaplist = [cmap(i) for i in range(cmap.N)] 
cmaplist[-1] = (1.0, 1.0, 1.0, 1.0) 
cmap = colors.LinearSegmentedColormap.from_list("mcm", cmaplist, cmap.N) 
random_amplitudes = np.random.rand(np.max(uniq) +1) 
random_amplitudes[0] = 1.0 

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(
        random_amplitudes[annotation[:, :, sagittal_slice]].T,
        interpolation="nearest",
        cmap=cmap)
plt.axis('off')
plt.xlim([0, annotation.shape[0]])
plt.ylim([annotation.shape[1], 0])
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "annotations.png"), dpi=200, facecolor="white")

Plotting the AIBS average brain for comparison:
http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_25.nrrd

In [ ]:
avg_brain = VoxelData.load_nrrd(join(DATA_FOLDER, "average_template_25.nrrd")).raw

In [ ]:
cmap = plt.cm.gray
norm = plt.Normalize(avg_brain.min(), avg_brain.max())
rgba = cmap(1-norm(avg_brain[:, :, sagittal_slice].T))

rgba-=0.1
rgba = np.maximum(rgba/np.max(rgba), 0)

fig = plt.figure(figsize=(15, 9.1))
plt.imshow(rgba, interpolation='nearest')
plt.axis('off')
plt.xlim([0, annotation.shape[0]])
plt.ylim([annotation.shape[1], 0])
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "avg_brain_slice.png"), dpi=200, facecolor="white")